import relevant packages, check working directory

In [126]:
import numpy as np
import os
import os.path as op
import pandas as pd
!pwd
#import utils as pu
#import tools


/media/joint-storage/puget/puget


In [138]:
# The default data directory will be '../../data'
# This can be redefined by the user of the function.
    
def get_snohomish_data():
    ''' Reads in the Snohomish .csv file.

        Parameters
        ----------
        none
        ----------

        Returns
        ----------
        dataframe with complete Snohomish data
    '''
    # Read the Snohomish file
    #from puget.data import DATA_PATH
    #SNOHOMISH_DATA = op.join(DATA_PATH, 'snohomish')
    #data_dir = op.join('..', '..', 'data', 'snohomish')
    # for final version
    #DATA_PATH = op.join('..', '..', 'data')
    #SNOHOMISH_DATA = op.join(DATA_PATH, 'snohomish')
    DATA_PATH = op.join('data')
    SNOHOMISH_DATA = op.join(DATA_PATH, 'snohomish')
    filename = op.join(SNOHOMISH_DATA,'Snohomish_HMIS_Data_Export_06152015.xlsx')
    xl = pd.ExcelFile(filename)
    xl.sheet_names
    df = xl.parse('Query2')
    return df


In [17]:
snohomish_df=get_snohomish_data()

use numpy's np.isnan to clean up columns. instead of passing a lambda, create a function that finds if the column is a float. use try...except to catch columns that are not floats

In [29]:
def nan_fx(x):
    try:
        val=np.sum(np.isnan(x))
    except TypeError:
        val = 0
    return val

In [23]:
f = lambda x: x + 1

In [25]:
f(1)

2

look at the general shape of data

In [141]:
snohomish_df.shape
#snohomish_df.describe()

(31008, 110)

create counts of non missing observations in each column

In [71]:
#row location 0 is the count of each column
column_counts=snohomish_df.describe().iloc[0]
#column_counts

only select columns where at least 150 observations have responses  
must keep 'DestinationAtExit' column despite missingness

In [221]:
#snohomish_df['DestinationAtExit'].describe()
destination=snohomish_df.filter(['DestinationAtExit', 'EnrollID'])
#destination.head()
good_counts=np.where(column_counts>150)

create a new data frame that is only the columns with > 150 observations

In [44]:
snohomish_newdf = snohomish_df[snohomish_df.columns[good_counts]]
snohomish_newdf.shape

create a list of columns that have float responses to further subset the new data frame

In [100]:
float_index=snohomish_newdf.dtypes =='float64'
float_columns=snohomish_newdf.columns[float_index]
snohomish_float=snohomish_newdf[float_columns]
snohomish_float.shape

apply nan_fx to count all non-missing row counts in the snohomish float data frame

In [157]:
#mc = missing counts
mc=snohomish_float.apply(nan_fx,axis=0)
#mc
final_columns = np.where(mc<snohomish_float.shape[0]*.05)
final_columns
snohomish_final=snohomish_float[snohomish_float.columns[final_columns]]
snohomish_final.shape

(31008, 13)

In [291]:
destination.head()

,DestinationAtExit,EnrollID
0,"Staying or living with friends, temporary tenu...",3468
1,"Rental by client, VASH Subsidy",24464
2,"Rental by client, other (non-VASH) ongoing hou...",687
3,"Rental by client, no ongoing housing subsidy",33974
4,"Rental by client, other (non-VASH) ongoing hou...",690


In [307]:
snohomish_merge = snohomish_final.join(destination, how= 'outer', lsuffix='_l', rsuffix='_r')
snohomish_merge.head()
#snohomish_merge.shape
#len(np.unique(snohomish_merge['EnrollID_l']))

,ClientID,EnrollID_l,FamilyID,NumberFamilyMembers,BirthDateQuality,Race_AmIndian,Race_Asian,Race_Black,Race_Hawaiian_PI,Race_White,CaseID,ChildrenInCase,AdultsInCase,DestinationAtExit,EnrollID_r
0,708,3468,487,1,NaN,0,0,0,0,1,2437,0,1,"Staying or living with friends, temporary tenu...",3468
1,716,24464,495,1,1,0,0,0,0,1,15968,0,1,"Rental by client, VASH Subsidy",24464
2,717,687,496,2,1,0,0,0,0,1,469,0,1,"Rental by client, other (non-VASH) ongoing hou...",687
3,717,33974,496,2,1,0,0,0,0,1,23247,0,2,"Rental by client, no ongoing housing subsidy",33974
4,718,690,497,1,1,0,0,0,0,1,472,0,1,"Rental by client, other (non-VASH) ongoing hou...",690


In [281]:
snohomish_df['DestinationAtExit'].describe()
#snohomish_final.describe()
#len(np.unique(snohomish_final['EnrollID']))

count          27012
unique            27
top       Don't Know
freq            8173
Name: DestinationAtExit, dtype: object

tweak exit function to take strings of exits/successes 

In [311]:
destination_map(snohomish_merge['DestinationAtExit'])

0       NaN
1       NaN
2       NaN
3        10
4       NaN
5       NaN
6       NaN
7       NaN
8       NaN
9       NaN
10      NaN
11        3
12        3
13        3
14        3
15      NaN
16      NaN
17      NaN
18      NaN
19      NaN
20      NaN
21      NaN
22      NaN
23       10
24      NaN
25      NaN
26       23
27      NaN
28      NaN
29        3
         ..
30978   NaN
30979   NaN
30980   NaN
30981   NaN
30982   NaN
30983   NaN
30984   NaN
30985   NaN
30986   NaN
30987   NaN
30988   NaN
30989   NaN
30990   NaN
30991   NaN
30992   NaN
30993   NaN
30994   NaN
30995   NaN
30996   NaN
30997   NaN
30998   NaN
30999   NaN
31000   NaN
31001   NaN
31002   NaN
31003   NaN
31004   NaN
31005   NaN
31006   NaN
31007   NaN
Name: DestinationAtExit, dtype: float64

In [359]:
data_path = os.getcwd()
map_table = 'destination_mappings.csv'
#df=pd.DataFrame(np.nan, index = np.arange(1,data.shape[0],1), columns = ['enrollID','string','success', 'subsidy'])
mapping_table = pd.read_csv(op.join(data_path,'data','metadata', map_table))
mapping_table = mapping_table[mapping_table.Standard == "New Standards"]
# Recode Subsidy column to boolean
mapping_table['Subsidy'] = mapping_table['Subsidy'].map({'Yes': True,
                                                         'No': False})
# Drop columns we don't need
mapping_table = mapping_table.drop(['Standard'], axis=1)
#mapping_table.head()
output_df = pd.merge(left=snohomish_merge, right=mapping_table, how='left',
                         left_on='DestinationAtExit',
                         right_on='DestinationDescription')
output_df = output_df.drop('DestinationDescription', axis=1)

output_df.head()


,ClientID,EnrollID_l,FamilyID,NumberFamilyMembers,BirthDateQuality,Race_AmIndian,Race_Asian,Race_Black,Race_Hawaiian_PI,Race_White,CaseID,ChildrenInCase,AdultsInCase,DestinationAtExit,EnrollID_r,DestinationNumeric,DestinationGroup,DestinationSuccess,Subsidy
0,708,3468,487,1,NaN,0,0,0,0,1,2437,0,1,"Staying or living with friends, temporary tenu...",3468,13,Temporary,Other Exit,False
1,716,24464,495,1,1,0,0,0,0,1,15968,0,1,"Rental by client, VASH Subsidy",24464,NaN,NaN,NaN,NaN
2,717,687,496,2,1,0,0,0,0,1,469,0,1,"Rental by client, other (non-VASH) ongoing hou...",687,NaN,NaN,NaN,NaN
3,717,33974,496,2,1,0,0,0,0,1,23247,0,2,"Rental by client, no ongoing housing subsidy",33974,10,Permanent,Successful Exit,False
4,718,690,497,1,1,0,0,0,0,1,472,0,1,"Rental by client, other (non-VASH) ongoing hou...",690,NaN,NaN,NaN,NaN


In [360]:
def destination_map(data, map_table = 'destination_mappings.csv'):
        data_path = os.getcwd()
        mapping_table = pd.read_csv(op.join(data_path,'data','metadata', map_table))
        mapping_table = mapping_table[mapping_table.Standard == "New Standards"]
        # Recode Subsidy column to boolean
        mapping_table['Subsidy'] = mapping_table['Subsidy'].map({'Yes': True,
                                                                 'No': False})
        # Drop columns we don't need
        mapping_table = mapping_table.drop(['Standard'], axis=1)

        # Merge the Destination mapping with the df
        # based on the last_destination string
        output_df = pd.merge(left=data, right=mapping_table, how='left',
                         left_on='DestinationAtExit',
                         right_on='DestinationDescription')

        output_df = output_df.drop('DestinationDescription', axis=1)

        return output_df

In [338]:
!pwd

/media/joint-storage/puget/puget


In [310]:
 def destination_map(column):
        df=pd.DataFrame(np.nan, index = np.arange(1,snohomish_merge.shape[0],1), columns = ['string','success', 'subsidy'])
        df['string']=column
        new_column = column.map(
        {'Emergency shelter, including hotel or motel paid for with emergency shelter voucher':1,
         'Transitional housing for homeless persons (including homeless youth)':2,
         'Permanent housing for formerly homeless persons (such as: CoC project; or HUD legacy programs; or HOPWA PH)':3,
         'Psychiatric hospital or other psychiatric facility':4,
         'Substance abuse treatment facility or detox center':5,
         'Hospital or other residential non-psychiatric medical facility':6,
         'Jail, prison or juvenile detention facility':7,
         'Rental by client, no ongoing housing subsidy':10,
         'Owned by client, no ongoing housing subsidy':11,
         'Staying or living with family, temporary tenure (e.g. room, apartment or house)':12,
         'Staying or living with friends, temporary tenure (e.g. room apartment or house)':13,
         'Hotel or motel paid for without emergency shelter voucher':15,
         'Foster care home or foster care group home':15,
         'Place not meant for habitation (e.g., a vehicle, an abandoned building, bus/train/subway station/airport or anywhere outside)':16,
         'Other':17,
         'Safe Haven':18,
         'Rental by client, with VASH housing subsidy':19,
         'Rental by client, with other ongoing housing subsidy':20,
         'Owned by client, with ongoing housing subsidy':21,
         'Staying or living with family, permanent tenure':22,
         'Staying or living with friends, permanent tenure':23,
         'Deceased':24,
         'Long-term care facility or nursing home':25,
         'Moved from one HOPWA funded project to HOPWA PH':26,
         'Moved from one HOPWA funded project to HOPWA TH':27,
         'Rental by client, with GPD TIP housing subsidy':28,
         'Residential project or halfway house with no homeless criteria':29,
         'No exit interview completed':30})
        return new_column
    # string description, success column, subsidy column
        

In [15]:
def get_snohomish_families():
    ''' Reads in the Snohomish data, and keeps only rows that are families

   Parameters
    ----------
    file_dict : dict or string
        if a dict, keys should be years, values should be full path to files
        if a string, should be the filename of the .csv table and data_dir,
            paths and years parameters are required
    data_dir : string
        full path to general data folder (usually puget/data);
            not required if file_dict is a dictionary
    paths : list
        list of directories inside data_dir to look for csv files in;
            not required if file_dict is a dictionary
    years : list
        list of years to include, default is to include all years;
            not required if file_dict is a dictionary
    metadata_file : string
        name of json metadata file with lists of columns to use for
        deduplication, columns to drop, categorical and time-like columns
    df_destination_colname : string
        column containing the numeric destination codes
    Returns
    ----------
    dataframe with only clients that come from a household with multiple people (a family)
    '''
    df = get_snohomish_data()

    gb = df.groupby('FamilyID')
    more_than_one = lambda x:(len(np.unique(x['ClientID'])) > 1)
    families = gb.filter(more_than_one).copy()
    #returns dataframe with families with more than one person (may be a family of all children or all adults)

    #Remove data entry errors:
    for famid, fam in families.groupby('FamilyID'):
        for caseid, case in fam.groupby('CaseID'):
            if not caseid == fam.iloc[0]['CaseID'] :  #if not the same caseID as was identified in the fam table
                if case['ClientID'].nunique() == 1 : # case['EnrollDate'].iloc[0] == prv_case_EnrollDate) :
                #if this condition is true, this case is an error
                    families = families[families.CaseID != caseid]
                else:
                    pass

            else: #if it is the same caseID as was identified in the fam table
                pass


    families = recode_snohomish_data(families)

    return families


